OG Credit: https://colab.research.google.com/drive/1AVh9dOsG9DKzfK7gOFrJuitPIcLPqlbO?usp=sharing#scrollTo=g8ntQ9CzRQ-q

In [1]:
!nvidia-smi

Wed Aug 30 14:21:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -q langchain transformers sentence_transformers llama-index

In [3]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline


In [ ]:
class customLLM(LLM):
   # model_name = "google/flan-t5-large"
    # model_name = "ClueAI/ChatYuan-large-v1"
    model_name = "THUDM/chatglm2-6b"   # 定义与训练模型
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})
    # 创建pipeline

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]

    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())



Loading THUDM/chatglm2-6b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/THUDM/chatglm2-6b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading THUDM/chatglm2-6b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/THUDM/chatglm2-6b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading THUDM/chatglm2-6b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/THUDM/chatglm2-6b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# 生成文本的嵌入向量
hfemb = HuggingFaceEmbeddings()
# 与文本嵌入模型进行交互
embed_model = LangchainEmbedding(hfemb)

In [ ]:
!pip install PyPDF2

PDF Method: Put/change the online pdf link here:

In [ ]:
# !wget http://unec.edu.az/application/uploads/2014/12/pdf-sample.pdf -O sample.pdf
# !wget -U NoSuchBrowser/1.0 http://arxiv.org/pdf/2302.03803.pdf -O sample.pdf
# !wget -U NoSuchBrowser/1.0 https://arxiv.org/pdf/2303.17489.pdf -O sample.pdf
!wget https://aclanthology.org/2022.ccl-1.1.pdf -O sample.pdf

In [ ]:
# https://www.geeksforgeeks.org/extract-text-from-pdf-file-using-python/
# importing required modules
from PyPDF2 import PdfReader

# creating a pdf reader object
reader = PdfReader('sample.pdf')
text_pdf = ''

# printing number of pages in pdf file
# getting a specific->all page from the pdf file
# extracting text from page   # text = page.extract_text((0,90))
for i in range(len(reader.pages)):
  page = reader.pages[i]
  text_pdf += page.extract_text()

# print(text_pdf)

TEXT Method: Put/change the text here:

In [ ]:
text1='''未来城市拥有很多不同的功能，如智慧城市、智能交通系统、远程医疗系统、智能家居系统、智慧医疗系统等。未来城市还使用物联网技术，可以远程控制，使用远程操控和传感器来监控城市的发展，帮助他们解决各种城市问题。

未来的城市还采用互联网技术，可以让城市变得更智能，让城市变得更智慧，让每一个人都能更轻松地生活。未来城市的智能设备也可以让城市的使用者更方便地掌握他们的生活，比如智能的智能交通系统和远程医疗系统和智能的安防系统等，可以提供他们更多的便利。
总之，未来城市的愿景是，让所有城市的使用者都能拥有一个美好的未来，让他们可以在自己的城市享受美好的生活。'''

In [ ]:
#documents = SimpleDirectoryReader('data').load_data()

from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]

text_list_pdf = [text_pdf]

documents_pdf = [Document(t) for t in text_list_pdf]

In [ ]:
# # set number of output tokens
# num_output = 500
# # set maximum input size
# max_input_size = 512
# # set maximum chunk overlap
# max_chunk_overlap = 15


# prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [ ]:
#index = GPTVectorStoreIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
index_pdf = GPTVectorStoreIndex.from_documents(documents_pdf, service_context=service_context)




In [ ]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

TEXT method A&Q:

In [ ]:
response = index.query( "未来城市可以做什么？")


In [ ]:
response.response

In [ ]:
response = index.query( "What's the main idea？")

In [ ]:
response.response

In [ ]:
response = index.query( "请描绘一幅画")

In [ ]:
response.response

PDF method A&Q:

In [ ]:
response_pdf = index_pdf.query( "这篇文章的作者是谁？")

In [ ]:
response_pdf.response

In [ ]:
response_pdf = index_pdf.query( "这篇文章讲了什么？")

In [ ]:
response_pdf.response

In [ ]:
response_pdf = index_pdf.query( "词汇如何变化？")

In [ ]:
response_pdf.response

In [ ]:
response_pdf = index_pdf.query( "请写一首诗")

In [ ]:
response_pdf.response

In [ ]:
# temp fix for running shell commands on Google Colab

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install gradio -q

In [ ]:
import gradio as gr

In [ ]:
index = None

In [ ]:
def build_the_bot(input_text):
  text_list = [input_text]
  documents = [Document(t) for t in text_list]
  global index
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
  return('Index saved successfull!!!')

In [ ]:

def chat(chat_history, user_input):

  bot_response = index.query(user_input)
  #print(bot_response)
  response = ""
  for letter in ''.join(bot_response.response): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]

In [ ]:

with gr.Blocks() as demo:
    gr.Markdown('# Q&A Bot with Hugging Face Models')
    with gr.Tab("Input Text Document"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Build the Bot!!!")
        text_button.click(build_the_bot, text_input, text_output)
    with gr.Tab("Knowledge Bot"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch(debug = True)